In [1]:
#This code is for process PCAWG ICGC consensus somatic pass only maf files. The output files are inputs for MutSigCV
#Author: Yiyun Rao
#Date last edit:05222020

In [2]:
from optparse import OptionParser
import shutil
import os
import re
import time

In [2]:
def parse_arguments():
    parser = OptionParser("Usage: %prog -i <inputpath>")
    parser.add_option("-m", "--maf_file",
                      dest="inputPath",
                      help="The full absolute path of the PCAWG ICGC consensus passonly maf file")
    
    (options, arg) = parser.parse_args()
    if not options.inputPath:
        parser.error("Requires at least one input file, the path of this script. Run with --help for help.")
        quit(1)
    return options

In [ ]:
file_maf = open("final_consensus_passonly.snv_mnv_indel.icgc.controlled.maf")

In [6]:
#def search_tumor_type_column(maffile):
start1 = time.time()

###Read the first line and find tumor type column(project_code)
first_line = file_maf.readline()
line_split = first_line.split("\t")
idx = [line_split.index(cols) for cols in line_split if re.findall("project_code",cols,flags = re.IGNORECASE)]
if idx:
    if len(idx)==1:
        tumor_col = int(idx[0]) 
    else:
        raise TumorTypeNotFoundError("MULTIPLE COLUMNS WERE MAPPED")
else:
    raise TumorTypeNotFoundError("TUMOR TYPE COLUMN NOT FOUND")

###Get current directory and create tumor-type directory
cwd = os.getcwd()
new_path = cwd + "/Tumor_type_maf"
if not os.path.exists(new_path):
    os.makedirs(new_path)
else:
    raise PathAlreadyExistsError("DIRECTORY ALREADY EXISTS")

# ###Get the list of tumor types
# list_tumor_type = []
# next(file_maf)
# for lines in file_maf:
#     line_split = lines.split("\t")
#     tumor_type = line_split[tumor_col]
#     if tumor_type not in list_tumor_type:
#         list_tumor_type.append(tumor_type)
# n_tumor_type = len(list_tumor_type)

# print str(n_tumor_type) + " EXIST."
# print "START PARSING TUMOR TYPE..."


###Parse the MAF file, create new file according to tumor type and put corresponding patients in it
next(file_maf)
for lines in file_maf:
    line_split = lines.split("\t")
    tumor_type = line_split[tumor_col]
    file_name = "ICGC_consensus_passonly_snv_mnv_indel_"+str(tumor_type)+"_05252020.maf"
    #IF file doesn't exist, create file and write the header line
    if not os.path.exists(new_path+'/'+file_name):
        with open(new_path+'/'+file_name,'w') as f:
            f.write(first_line)
        f.close()
    #Append the file to 
    with open(new_path+'/'+file_name,"a") as f1:
        f1.write(lines)
    f1.close()

end1 = time.time()
print "TIME USED "+str(end1-start1) + " seconds"

TIME USED 1659.85083008 seconds


In [3]:
#Main function area
if __name__ == "__main__":
    arguments = parse_arguments()
    maf_file = arguments.inputPath
    start1 = time.time()
    search_tumor_type_column(maf_file)
    end1 = time.time()

Usage: ipykernel_launcher.py -i <inputpath>

ipykernel_launcher.py: error: no such option: -f


SystemExit: 2

/opt/aci/sw/python/2.7.14_anaconda-5.0.1/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
#Error Area
class TumorTypeNotFoundError(Exception):
    def __init__(self, *args):
        if args:
            self.message = args[0]
        else:
            self.message = None

    def __str__(self):
        if self.message:
            return 'TumorTypeNotFoundError, {0} '.format(self.message)
        else:
            return 'TumorTypeNotFoundError has been raised'
        
class PathAlreadyExistsError(Exception):
    def __init__(self, *args):
        if args:
            self.message = args[0]
        else:
            self.message = None

    def __str__(self):
        if self.message:
            return 'PathAlreadyExists, {0} '.format(self.message)
        else:
            return 'PathAlreadyExists has been raised'